In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
import anndata as ad
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
print(sys.executable)
from matplotlib.colors import Normalize

def make_df_annotation_azimut1_5_scanvi_v2(df):
    df['cluster_azimut1_5_scanvi_v2'] = df['cluster_azimut1_5_scanvi'].replace(
    {'NK': 'NK+Proliferating', 'NK Proliferating': 'NK+Proliferating'})
    return df

In [ ]:
pre = "MH111"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)

### Cyno

In [ ]:
species = 'cyno'

In [ ]:
_,_,_,base_anndata_objects_M24 = h.return_local_paths(drive,pre='M24',add_path = False)
adata = sc.read_h5ad(os.path.join(base_anndata_objects_M24,'M24' + '_' + species + '_anno_celltypes_v0.h5ad'))

- take background corrected counts
- keep all genes 
- filter for QC cells
- building analogous to M23 in first part
- take cell type 

In [ ]:
species = 'cyno'
warnings.filterwarnings("ignore")
IDs = h.return_cellranger_IDs(species)
adata_all_cyno = h.read_bg_corrected_counts(drive,cellranger_IDs = IDs,pre_bg_correction = 'M05', species = species)
#add MT nomenclature
cynoanno = pd.read_csv(os.path.join(h.return_git_path_local(),"preprocessing","ensembl","mart_export.txt.gz"), compression='gzip')
mt_genes_cyno = h.return_mt_genes_cyno(cynoanno)
adata_all_cyno = h.add_MT_prefix(adata=adata_all_cyno,mt_genes=mt_genes_cyno)

- load in QC

In [ ]:
_,base_table_path_M21,_,_ = h.return_local_paths(drive = drive,pre = "M21",add_path = False)

In [ ]:
anno_cyno = pd.read_csv((os.path.join(base_table_path_M21,'M21_'  + species + '_anno_QC.csv')),index_col=0)

In [ ]:
adata_all_cyno = h.add_anno_to_adata(adata=adata_all_cyno,
                  anno_df=anno_cyno,
                  anno_columns_to_add=anno_cyno.columns,
                  rsuffix='_R')

In [ ]:
adata_all_cyno = h.filter_adata_obs(adata_all_cyno,col_name='QC_summary',val='passed_QC')
#adata_all_cyno.obs = adata_all_cyno.obs.loc[:, ~adata_all_cyno.obs.columns.duplicated()]

In [ ]:
scanpy_genes = pd.DataFrame(adata_all_cyno.var.index,columns = ['scanpy_gene'])
scanpy_genes.to_csv(os.path.join(base_table_path,pre + '_scanpy_genes.csv'))

- add annotation

In [ ]:
path_clusteranno_s0140 = 'F:\\monkey_IZI\\analysis\\tables\\s0140_clusteranno_scvi.txt.gz'
clusteranno_s0140 = pd.read_csv(path_clusteranno_s0140,compression = 'gzip',index_col=0)

In [ ]:
ortho_tab_s0135_nm_path = os.path.join(*["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysisR",
 "results_GIT","s0135_cyno2human_n_to_m_orthologues_unfilteredApes.csv.gz"])
df_ortho_tab_s0135_nm = pd.read_csv(ortho_tab_s0135_nm_path, compression='gzip')

In [ ]:
adata_duplicate_list = []
original_index = adata_all_cyno.var.index.copy()  

for cyno_gene in original_index:
    if cyno_gene.endswith('-1'):
        tab = df_ortho_tab_s0135_nm[df_ortho_tab_s0135_nm['seurat_gene'] == cyno_gene.split('-1')[0]]
    else:
        tab = df_ortho_tab_s0135_nm[df_ortho_tab_s0135_nm['seurat_gene'] == cyno_gene]
    gene_idx = list(original_index).index(cyno_gene)  
    human_gene_names = tab[tab['Human gene name'].notna()]['Human gene name'].values
    
    for q, human_gene_name in enumerate(human_gene_names):
        if q == 0:
            # Modify the main AnnData index for the first human gene name
            ind_list = list(adata_all_cyno.var.index)
            ind_list[gene_idx] = human_gene_name
            adata_all_cyno.var.index = ind_list
        else:
            # Create a copy of the AnnData object for additional human gene names
            single_gene_anndata = adata_all_cyno[:, gene_idx].copy()
            single_gene_anndata.var.index = [human_gene_name]
            adata_duplicate_list.append(single_gene_anndata)

In [ ]:
adata_duplicates = ad.concat(adata_duplicate_list,axis=1)

In [ ]:
adata_cyno_all = ad.concat([adata_duplicates,adata_all_cyno],axis=1)

In [ ]:
gene_names = adata_cyno_all.var.index.values
unique_genes, counts = np.unique(gene_names, return_counts=True)
genes_appear_once = unique_genes[counts == 1]

In [ ]:
genes_to_sum = set(adata_cyno_all.var.index.values) - set(genes_appear_once)

In [ ]:
list_adatas_to_sum = []
for gene in genes_to_sum:
    adata_to_sum = adata_cyno_all[:,adata_cyno_all.var.index == gene]
    sum_from_duplicates = np.sum(pd.DataFrame.sparse.from_spmatrix(adata_to_sum.X),axis=1).values
    sum_gene_anndata = adata_to_sum [:, 0].copy()
    sum_gene_anndata.X = sum_from_duplicates
    sum_gene_anndata.layers={}
    list_adatas_to_sum.append(sum_gene_anndata)

In [ ]:
adatas_duplicates_summed = ad.concat(list_adatas_to_sum,axis=1)
adata_single_genes = adata_cyno_all[:,adata_cyno_all.var.index.isin(genes_appear_once)]

adata_cyno_final = ad.concat([adatas_duplicates_summed,adata_single_genes],axis=1)

adata_cyno_final.obs = pd.merge(adata_cyno_final.obs, clusteranno_s0140, left_index=True, right_index=True)

adata_cyno_final.write(os.path.join(base_anndata_objects,pre + '_adata_cyno_final_CCC.h5ad'))

### Human

In [ ]:
species = 'human'
#human
_,base_table_path_H21,_,_ = h.return_local_paths(drive = drive,pre = "H21",add_path = False)

In [ ]:
#human
species = 'human'
warnings.filterwarnings("ignore")
IDs = h.return_cellranger_IDs(species)
adata_all_human = h.read_bg_corrected_counts(drive,cellranger_IDs = IDs,pre_bg_correction = 'H05_01', species = species)

In [ ]:
anno_human = pd.read_csv((os.path.join(base_table_path_H21,'H21_'  + species + '_anno_QC.csv')),index_col=0)

In [ ]:
adata_all_human = h.add_anno_to_adata(adata=adata_all_human,
                  anno_df=anno_human,
                  anno_columns_to_add=anno_human.columns,
                  rsuffix='_R')

In [ ]:
scanpy_genes = pd.DataFrame(adata_all_human.var.index,columns = ['scanpy_gene'])
scanpy_genes.to_csv(os.path.join(base_table_path,pre + 'human_scanpy_genes.csv'))

In [ ]:
adata_all_human = h.filter_adata_obs(adata_all_human,col_name='QC_summary',val='passed_QC')

In [ ]:
adata_all_human.obs = pd.DataFrame(index=adata_all_human.obs.index)

In [ ]:
adata_all_human.obs = pd.merge(adata_all_human.obs, clusteranno_s0140, left_index=True, right_index=True)

In [ ]:
adata_all_human.write(os.path.join(base_anndata_objects,pre + '_adata_human_final_CCC.h5ad'))

## Save session

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()